<a href="https://colab.research.google.com/github/aaronbbarclay/mine/blob/master/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip show tensorflow
!pip show tensorboard
!pip show python


Name: tensorflow
Version: 1.13.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: six, wheel, absl-py, tensorboard, keras-preprocessing, numpy, astor, termcolor, protobuf, grpcio, tensorflow-estimator, keras-applications, gast
Required-by: stable-baselines, magenta, fancyimpute
Name: tensorboard
Version: 1.13.1
Summary: TensorBoard lets you watch Tensors Flow
Home-page: https://github.com/tensorflow/tensorboard
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: wheel, markdown, numpy, werkzeug, grpcio, protobuf, absl-py, six
Required-by: tensorflow


In [2]:
%cd
!git clone --quiet https://github.com/tensorflow/models.git
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive six

/root
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-tk is already the newest version (2.7.15~rc1-1).
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 7 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,688 kB of additional disk space will be used.
Get:1 

In [3]:
# Compile protocol buffers
%cd ~/models/research
!protoc object_detection/protos/*.proto --python_out=.

/root/models/research


In [0]:
# Configure env variables
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/'

In [5]:
%cd ~/models/research/

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

sys.path.append("/models/research/")
from object_detection.utils import ops as utils_ops

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


/root/models/research


In [6]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
 
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
 
PATH_TO_LABELS = os.path.join('/root/models/research/object_detection/data', 'mscoco_label_map.pbtxt')
print(PATH_TO_LABELS)
 
NUM_CLASSES = 90

/root/models/research/object_detection/data/mscoco_label_map.pbtxt


In [7]:
%cd /root/models/research/object_detection/data
%ls

/root/models/research/object_detection/data
ava_label_map_v2.1.pbtxt
face_label_map.pbtxt
fgvc_2854_classes_label_map.pbtxt
kitti_label_map.pbtxt
mscoco_complete_label_map.pbtxt
mscoco_label_map.pbtxt
mscoco_minival_ids.txt
oid_bbox_trainable_label_map.pbtxt
oid_object_detection_challenge_500_label_map.pbtxt
oid_v4_label_map.pbtxt
pascal_label_map.pbtxt
pet_label_map.pbtxt


In [8]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())
 
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
print(PATH_TO_CKPT)

ssd_mobilenet_v1_coco_2017_11_17/frozen_inference_graph.pb


In [0]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

print(label_map)

In [107]:
%mkdir ~/data
%cd ~/data
!rm -rf /root/data/mine/
!git clone https://github.com/aaronbbarclay/mine
%cd /root/data/mine/machineLearning/macbethIdentify/data/
%ls
!unzip macbeth_identify.zip

mkdir: cannot create directory ‘/root/data’: File exists
/root/data
Cloning into 'mine'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 77 (delta 43), reused 9 (delta 0), pack-reused 0
Unpacking objects: 100% (77/77), done.
/root/data/mine/machineLearning/macbethIdentify/data
macbeth_identify.zip*
Archive:  macbeth_identify.zip
   creating: macbeth_identify/
  inflating: macbeth_identify/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/macbeth_identify/
  inflating: __MACOSX/macbeth_identify/._.DS_Store  
  inflating: macbeth_identify/label_map.pbtxt  
  inflating: __MACOSX/macbeth_identify/._label_map.pbtxt  
  inflating: macbeth_identify/train.txt  
  inflating: __MACOSX/macbeth_identify/._train.txt  
   creating: macbeth_identify/img/
  inflating: macbeth_identify/img/IMG_1336.jpg  
   creating: __MACOSX/macbeth_identify/img/
  inflating: __MACOSX/macbeth_identify/img/._IMG

In [106]:

%ls
%cd /root/data/mine/machineLearning/macbethIdentify/data/
%ls

img.zip  mine/
/root/data/mine/machineLearning/macbethIdentify/data
macbeth_identify.zip*


In [11]:
%cd ~/data/mine

imageFilesBasePath = PATH_TO_TEST_IMAGES_DIR = '/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img'
annotationsFilesBasePath = '/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/annotations/'
labelMapPath = '/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/label_map.pbtxt'
testPath = '/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/test.txt'
trainPath = '/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/train.txt'
valPath = '/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/val.txt'



%cd /root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img

/root/data/mine
/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img


In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [0]:
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 8) ]
TEST_IMAGE_PATHS = [ os.path.join(imageFilesBasePath, x) for x in os.listdir(imageFilesBasePath) ]
print(TEST_IMAGE_PATHS)

for img in TEST_IMAGE_PATHS:
  assert os.path.isfile(img)
  print(img)


In [0]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
        
      #print(output_dict)
        
  return output_dict

In [0]:
from PIL import Image

def resize_image(image_path=None):
  #imageDest = Image.new("RGB", (640, 480))
  resizeImagePath = image_path.replace("/img/", "/img_coco/")
  if not os.path.isdir(os.path.dirname(resizeImagePath)):
    os.mkdir(os.path.dirname(resizeImagePath))
  
  imageSrc = Image.open(image_path)
  imageDest = imageSrc.resize((640, 480), Image.LANCZOS)
  
  return imageDest
    


In [0]:
def write_image():
  #

In [109]:
IMAGE_SIZE = (12, 8)
from google.colab import files
import imageio
import numpy

for image_path in TEST_IMAGE_PATHS[:5]:
  if not image_path.endswith(".jpg"):
    continue
  saveImage = image_path.replace("/img/", "/img_labels/")

  #print(image_path)
  #image = Image.open(image_path)
  image = resize_image(image_path)
  imageArray = numpy.array(image)
  #print("Image Dimensions: {} {}".format(image.width, image.height))
  print(image_path)
  #print(detection_graph)
  
  #imageio.imwrite(saveImage, imageArray)
  #files.download(saveImage)
  

  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)

  
  
  #saveImage2 = saveImage.replace(".jpg", "__2.jpg")
  #print(saveImage2)
  
  #print(image_np_expanded.shape)
  #print(image_np_expanded[0])
  img = Image.fromarray(image_np_expanded[0], 'RGB')
  #im = Image.fromarray(np.uint8(image_np_expanded))
  #img.save(saveImage2)  
  #files.download(saveImage2)
  
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  #print(output_dict)
  if output_dict is None:
    continue
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  
  #plt.figure(figsize=IMAGE_SIZE)
  #plt.imshow(image_np_expanded)
  

  if not os.path.isdir(os.path.dirname(saveImage)):
    os.mkdir(os.path.dirname(saveImage))
    
  saveImage3 = saveImage.replace(".jpg", "__3.jpg")
  
  img = Image.fromarray(image_np_expanded[0], 'RGB')
  img.save(saveImage3)  
  print(saveImage3)
  #files.download(saveImage3)
  
  #imageio.imwrite(saveImage, image_np_expanded[:, :, 3])

  #img = Image.fromarray(image_np_expanded, 'RGB')
  #img.save(saveImage)
  #files.download(saveImage)
  
  

/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img/sg11.jpg
/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img_labels/sg11__3.jpg
/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img/IMG_1330.jpg
/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img_labels/IMG_1330__3.jpg
/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img/IMG_1319.jpg
/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img_labels/IMG_1319__3.jpg
/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img/catherine_mary_stewart_00331-mos.jpg
/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img_labels/catherine_mary_stewart_00331-mos__3.jpg
/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img/IMG_1347.jpg
/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img_labels/IMG_1347__3.jpg


In [83]:
%cd /root/data/img.zip /root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img_labels/
%ls -l
%pwd

[Errno 2] No such file or directory: '/root/data/img.zip /root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img_labels/'
/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img
total 0


'/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img'

In [113]:
!rm -rf /root/data/img.zip
!zip -r /root/data/img.zip /root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img_labels/



  adding: root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img_labels/ (stored 0%)
  adding: root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img_labels/IMG_1330__3.jpg (deflated 2%)
  adding: root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img_labels/catherine_mary_stewart_00331-mos__3.jpg (deflated 4%)
  adding: root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img_labels/IMG_1319__3.jpg (deflated 1%)
  adding: root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img_labels/sg11__3.jpg (deflated 3%)
  adding: root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img_labels/IMG_1347__3.jpg (deflated 1%)


In [0]:
files.download("/root/data/img.zip")

In [114]:
ls -lh /root/data/

total 212K
-rw-r--r-- 1 root root 207K May 23 11:31 img.zip
drwxr-xr-x 4 root root 4.0K May 23 11:26 mine/
